In [7]:
import getpass
import os
import dotenv

dotenv.load_dotenv()

True

In [8]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o-mini", model_provider="openai")


In [ ]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
    Analyze the following code and identify security vulnerabilities.
Return the output in JSON format with the following fields:
- "line": The line number where the issue occurs.
- "token": The specific code token that is problematic.
- "problem": A short name for the issue.
- "explanation": A brief explanation of why this is a security risk.

Code:
{code}
"""
)

In [10]:
code_snippet = '''
function withdraw(uint amount) public{
    if (credit[msg.sender]>= amount) {
      require(msg.sender.call.value(amount)());
      // here we are updating the credit after the external call, which is vulnerable to reentrancy attack
      credit[msg.sender]-=amount;
    }
}
'''

In [11]:
from langchain.chains import LLMChain
chain = chain = prompt | model

In [12]:
result = chain.invoke({"code": code_snippet})
result

AIMessage(content='```json\n[\n    {\n        "line": 5,\n        "token": "msg.sender.call.value(amount)()",\n        "problem": "Reentrancy Attack",\n        "explanation": "This code calls an external contract before updating the state variable \'credit\'. This can allow the called contract to reenter the \'withdraw\' function and manipulate state variables before they are updated, leading to unauthorized fund withdrawals."\n    }\n]\n```', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 139, 'total_tokens': 229, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_9654a743ed', 'id': 'chatcmpl-BHS7qflJGuLSJcZmAhSxHadjFBy1o', 'finish_reason': 'stop', 'logprobs': None}, id='run-2e9aba1c-c6df-4d87-9ade-5ff